In [1]:
from data_processing.dataset import FaceDataset
import pytorch_lightning as L
from model.gauss_predictor import GaussPredictor
from model.gauss_predictor_with_latent_codes import GaussPredictorWithLatentCodes
from data_processing.data_module import FaceDataModule
from pytorch_lightning.loggers import WandbLogger
import yaml
from datetime import datetime
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import time
import torch
from tqdm import tqdm
import argparse

config = yaml.safe_load(open('configs/settings_final_template_flame.yaml', 'r'))
config['data']['force_rebuild'] = True
config['data']['save_results'] = False

face_data_module = FaceDataModule(config=config)
face_data_module.setup('fit')

#gauss_predictor = GaussPredictor.load_from_checkpoint(config['model']['checkpoint'], config=config, data_module=face_data_module)
#gauss_predictor.eval()

with torch.no_grad():
    batch = face_data_module.val_dataloader().dataset[9]

    initial_gaussians = batch['initial_gaussians']
    images = batch['images']
    target_depths = batch['depths']
    sequence_id = batch['sequence_id']

    camera_params = batch['camera_params']
    camera_params['cam_2_world_poses'] = FaceDataset.dicts_to_poses(camera_params['cam_2_world_poses'])

    input_serials = batch['input_serials']
    input_cam_2_world_poses = {serial: cam_2_world_pose for serial, cam_2_world_pose in camera_params['cam_2_world_poses'].items() if serial in input_serials}
    input_camera_params = {'cam_2_world_poses': input_cam_2_world_poses, 'intrinsics': camera_params['intrinsics']}

    #predicted_gaussians = gauss_predictor.forward({key: val.to(gauss_predictor.device) for key, val in initial_gaussians.items()})

    gaussian_xyz_start = initial_gaussians['xyz'].cuda()
    #gaussian_xyz_pred = predicted_gaussians['xyz'].cuda()
    #gaussian_scales = predicted_gaussians['scaling'].cuda()
    #opacities = predicted_gaussians['opacity'].cuda()
    features = initial_gaussians['features'].cuda()

    #dists = (gaussian_xyz_pred - gaussian_xyz_start).square().sum(-1).sqrt()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/oroz/anaconda3/envs/gaussian_splatting/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/oroz/anaconda3/envs/gaussian_splatting/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


ENCODER: F
train self.face_ids ['017', '018', '024', '030', '031', '032', '033', '035', '036', '037', '038', '040', '042', '043', '055', '056', '057', '058', '060', '061', '063', '064', '065', '067', '068', '068', '069', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '085', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '102', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '162', '163', '164', '165', '166', '167', '168', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '191', '192', '193', '194', '195', '197', '198', '199', '200', '201', '2

In [2]:
#print(dists.min())
#print(dists.max())

#log_dists = dists.log1p()
#dists_np = dists.cpu().numpy()
#dists_np = np.log(dists_np)
# dists = (dists - dists.min()) / (dists.max() - dists.min())

In [3]:
import k3d
import numpy as np
import matplotlib.cm as cm

plot = k3d.plot(grid_auto_fit=False, grid_visible=False, height=1024)

#dists_np = features.cpu().numpy()[:, -3]
#dists_np = gaussian_scales.exp().cpu().numpy().sum(-1)
#dists_np = opacities.cpu().numpy().sum(-1)

#plot += k3d.points(gaussian_xyz_pred.cpu().numpy(), point_size=0.005, color_map=k3d.colormaps.matplotlib_color_maps.Viridis, attribute=dists_np, color_range=[0, 1])
plot += k3d.points(gaussian_xyz_start.cpu(), point_size=0.005, colors=[0x888888]*gaussian_xyz_start.shape[0])

plot.display()

Output()